# Steps towards developing a postgres dol

We're going to need a database to play with, so first let's get that! 

We don't like to show secrets in our code, so we'll use [config2py](https://pypi.org/project/config2py/) to specify our DB keys.

In [5]:
import config2py

POSTGRESS_TEST_DB_URL = config2py.config_getter('POSTGRESS_TEST_DB_URL')

## SQLAlchemy

We'll use [SQLAlchemy](https://pypi.org/project/SQLAlchemy/) to interface with postgress from python.

Let's first look at what this sqlalchemy interface looks like with our test table, 
doing some basic operations that we'll won't to DOLify later.

### get list of table names

In [4]:
from sqlalchemy import create_engine
from sqlalchemy.engine.reflection import Inspector

# Replace the placeholder values with your actual database credentials
engine = create_engine(POSTGRESS_TEST_DB_URL)

inspector = Inspector.from_engine(engine)

# List table names
table_names = inspector.get_table_names()
print(table_names)

/var/folders/mc/c070wfh51kxd9lft8dl74q1r0000gn/T/ipykernel_65051/1019456399.py:7: SADeprecationWarning: The from_engine() method on Inspector is deprecated and will be removed in a future release.  Please use the sqlalchemy.inspect() function on an Engine or Connection in order to acquire an Inspector. (deprecated since: 1.4)
  inspector = Inspector.from_engine(engine)


['app', 'prompt_template', 'users']


## Get rows for a given table and selector

In a given table, get a generator the will yield rows given a selector (selection logic), by default the "everything" selector, yielding all rows

In [7]:
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.orm import declarative_base, sessionmaker, Session

engine = create_engine(POSTGRESS_TEST_DB_URL)

SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

# Déclaration de la base pour la définition des modèles SQLAlchemy
Base = declarative_base()

class User(Base) :
    __tablename__ = "user"
    id = Column(Integer, primary_key=True)
    name = Column(String, index=True)
    token = Column(String, index=True)

class App(Base) :
    __tablename__ = "app"
    id = Column(Integer, primary_key=True, index=True)
    name = Column(String, index=True)

class AppPermission(Base) :
    __tablename__ = "app_permission"
    id = Column(Integer, primary_key=True, index=True)
    user_id = Column(Integer)
    app_id = Column(Integer)

class PromptTemplate(Base) :
    __tablename__ = "prompt_template"
    id = Column(Integer, primary_key=True)
    name = Column(String)
    template = Column(String)
    rjsf_ui = Column(String)

In [39]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy import select, text

Session = sessionmaker(bind=engine)
session = Session()

# Selection logic, by default fetching all
query = select(PromptTemplate).filter()  # Adjust the number per your needs

# for row in session.execute(query):
#     print(row)

cursor = session.execute(query)

print(f"{type(cursor)=}")

from typing import Iterator, Iterable, Generator
print(f"{isinstance(cursor, Generator)=}")
print(f"{isinstance(cursor, Iterator)=}")  # has __next__ method (so can do next(cursor))
print(f"{isinstance(cursor, Iterable)=}")  # has __iter__ method (so can do for x in cursor...)


type(cursor)=<class 'sqlalchemy.engine.result.ChunkedIteratorResult'>
isinstance(cursor, Generator)=False
isinstance(cursor, Iterator)=True
isinstance(cursor, Iterable)=True


In [42]:
from sqlalchemy import create_engine, Table, MetaData, select

metadata = MetaData()

# Reflect the table from the database
prompt_template_table = Table('prompt_template', metadata, autoload_with=engine)

# Now you can use this table object to construct a query
query = select(prompt_template_table)

# Execute the query
with engine.connect() as connection:
    result = connection.execute(query)
    for row in result:
        print(row)  # Each row is a RowProxy object that allows for column access by name


(1, 'Template1', 'Template content 1', '{"ui": "ui_content_1"}')
(2, 'Template2', 'Template content 2', '{"ui": "ui_content_2"}')
(3, 'Template3', 'Template content 3', '{"ui": "ui_content_3"}')
(4, 'Template1', 'Template content 1', '{"ui": "ui_content_1"}')
(5, 'Template2', 'Template content 2', '{"ui": "ui_content_2"}')
(6, 'Template3', 'Template content 3', '{"ui": "ui_content_3"}')


## My first (read only) DOL

In [95]:
from typing import Mapping, Sized, Iterable

class PostgresTableRows(Sized, Iterable):
    def __init__(self, engine, table_name):
        self.engine = engine
        self.table_name = table_name
        self.metadata = MetaData()
        self.table = Table(self.table_name, self.metadata, autoload_with=self.engine)

    def __iter__(self):
        query = select(self.table)
        with self.engine.connect() as connection:
            result = connection.execute(query)
            for row in result:
                yield row

    def __len__(self):
        query = select(self.table)
        with self.engine.connect() as connection:
            result = connection.execute(query)
            return result.rowcount
        
class PostgresBaseColumnsReader(Mapping):
    """Here, keys are column names and values are column values"""
    def __init__(self, engine, table_name):
        self.engine = engine
        self.table_name = table_name
        self.metadata = MetaData()
        self.table = Table(self.table_name, self.metadata, autoload_with=self.engine)
        
    def __iter__(self):
        return (column_obj.name for column_obj in self.table.columns)
    
    def __len__(self):
        return len(self.table.columns)
    
    def __getitem__(self, key):
        # TODO: Finish
        query = select(self.table).with_only_columns([self.table.c[key]])
        with self.engine.connect() as connection:
            result = connection.execute(query)
            return result.fetchall()
    

from typing import Callable  


class PostgresBaseKvReader(Mapping):
    """A mapping view of a table, 
    where keys are values from a key column and values are values from a value column.
    There's also a filter function that can be used to filter the rows.
    """
    def __init__(
            self, engine, table_name, 
            key_columns=None, 
            value_columns=None,
            filt=None
        ):
        self.engine = engine
        self.table_name = table_name
        self.metadata = MetaData()
        self.table = Table(self.table_name, self.metadata, autoload_with=self.engine)
        self.filt = filt

    def __iter__(self):
        query = select(self.table)
        with self.engine.connect() as connection:
            result = connection.execute(query)
            for row in result:
                yield row

    def __len__(self):
        query = select(self.table)
        with self.engine.connect() as connection:
            result = connection.execute(query)
            return result.rowcount
        
    def __getitem__(self, key):
        query = select(self.table)
        with self.engine.connect() as connection:
            result = connection.execute(query)
            return result.fetchall()
        
        
class PostgressTables(Mapping):
    def __init__(self, engine):
        self.engine = engine
        self.metadata = MetaData()
        self.metadata.reflect(bind=self.engine)

    def __getitem__(self, key):
        return PostgresBaseKvReader(self.engine, key)
        # Or do something with this:
        # return self.metadata.tables[key]

    def __iter__(self):
        return iter(self.metadata.tables)

    def __len__(self):
        return len(self.metadata.tables)


In [79]:
t = PostgresTableRows(engine, 'prompt_template')    
list(t)


[(1, 'Template1', 'Template content 1', '{"ui": "ui_content_1"}'),
 (2, 'Template2', 'Template content 2', '{"ui": "ui_content_2"}'),
 (3, 'Template3', 'Template content 3', '{"ui": "ui_content_3"}'),
 (4, 'Template1', 'Template content 1', '{"ui": "ui_content_1"}'),
 (5, 'Template2', 'Template content 2', '{"ui": "ui_content_2"}'),
 (6, 'Template3', 'Template content 3', '{"ui": "ui_content_3"}')]

In [ ]:
user, prompt_template_name, prompt_template_string, rjsf_ui_spec

key-value views
--> user_prompt_templates
--> user_rjsf_ui_spec

In [ ]:
from raglab.stores.simple_stores

In [ ]:
def PostgresBaseKvReader(Mapping):
    """A mapping view of a table, 
    where keys are values from a key column and values are values from a value column.
    There's also a filter function that can be used to filter the rows.
    """
    def __init__(
            self, engine, table_name, 
            key_column=None, 
            value_column=None, 
            filt=None

In [96]:
t = PostgresBaseColumnsReader(engine, 'prompt_template')
list(t)


['id', 'name', 'template', 'rjsf_ui']

In [ ]:
user_prompt_templates = MkUserPromptTemplateStore(user, ...)

list(user_prompt_templates)
user_prompt_templates['prompt_template_id']
user_prompt_templates['prompt_template_id'] = 'my prompt template'



In [110]:
# d = {'a': 1, 'b': 2}

# d.__getitem__('a')


# mk_app({user_store, methods=['__getitem__', '__setitem__']})

# ./user_store/prompt_templates/__getitem__?key=a

1

In [127]:
user, name, prompt_template_str, rjsf_ui_spec



NameError: name 'user' is not defined

In [59]:
tables = PostgressTables(engine)
list(tables)

['app', 'prompt_template', 'users']

In [51]:
t = tables['prompt_template']


In [54]:
list(t)

[(1, 'Template1', 'Template content 1', '{"ui": "ui_content_1"}'),
 (2, 'Template2', 'Template content 2', '{"ui": "ui_content_2"}'),
 (3, 'Template3', 'Template content 3', '{"ui": "ui_content_3"}'),
 (4, 'Template1', 'Template content 1', '{"ui": "ui_content_1"}'),
 (5, 'Template2', 'Template content 2', '{"ui": "ui_content_2"}'),
 (6, 'Template3', 'Template content 3', '{"ui": "ui_content_3"}')]

In [48]:
dir(t)

['__annotations__',
 '__bool__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__invert__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__visit_name__',
 '__weakref__',
 '_all_selected_columns',
 '_annotate',
 '_annotations',
 '_annotations_cache_key',
 '_anonymous_fromclause',
 '_assert_no_memoizations',
 '_autoincrement_column',
 '_autoload',
 '_cache_key_traversal',
 '_clone',
 '_cloned_set',
 '_cols_populated',
 '_columns',
 '_compile_w_cache',
 '_compiler',
 '_compiler_dispatch',
 '_constructor',
 '_copy_internals',
 '_de_clone',
 '_deannotate',
 '_dialect_kwargs_traverse_internals',
 '_execute_on_connection',
 '_execute_on_scal